<h1 style="text-align: center;">Trabajo Práctico N°4:</h1>
<h1 style="text-align: center;">Construcción y análisis de modelos de QSAR</h1>


## 🎯  Objetivos

* Aplicar los criterios de Unger y Hansch para construir, evaluar y seleccionar ecuaciones de QSAR.
* Aplicar herramientas estadísticas en modelos de regresión lineal simple y múltiple.
* Aprender a seleccionar descriptores relevantes en base a su correlación con la actividad biológica.
* Interpretar los coeficientes y métricas de ajuste del modelo (R², error estándar, etc.), y evaluar su capacidad predictiva.

## 🔍 Introducción a QSAR

El desarrollo racional de nuevos fármacos requiere cada vez más del uso de herramientas computacionales que permitan analizar, interpretar y predecir la actividad biológica de compuestos en función de su estructura química. Dentro de este marco, las **Relaciones Cuantitativas Estructura-Actividad** (QSAR, por sus siglas en inglés) constituyen una estrategia fundamental para correlacionar propiedades moleculares cuantificables con la eficacia o afinidad de una serie de compuestos frente a un blanco terapéutico común.

Para construir un modelo QSAR confiable, es necesario contar con un conjunto de moléculas adecuadamente seleccionadas: deben compartir un mismo mecanismo de acción, actuar sobre el mismo receptor, tener actividades reportadas en condiciones experimentales comparables y presentar una diversidad estructural razonable dentro de un marco común. Además, es imprescindible preparar los datos, evaluar sus distribuciones, normalizar sus escalas y seleccionar cuidadosamente los descriptores que formarán parte del modelo.

En este trabajo práctico, aplicaremos estos conceptos trabajando sobre un conjunto de agonistas que actúan sobre el receptor adrenérgico β2, que seleccionamos en el Seminario N°4. A través de diversas etapas, aprenderemos a construir modelos matemáticos que permitan correlacionar propiedades estructurales con la potencia biológica de los compuestos, evaluando el ajuste de dichos modelos y su utilidad predictiva. Para ello, retomaremos y aplicaremos las **reglas de Unger y Hansch**, que nos orientan sobre la forma, linealidad y relevancia de las variables utilizadas en modelos de este tipo.

## *Reglas de Unger y Hansch*

- 📊 <u>Selección de variables independientes</u>: los descriptores elegidos deben ser independientes entre sí; con coeficientes de correlación inferior a 0.6-0.7.
- ⚖️ <u>Justificación número de términos o variables</u>: para evitar correlación casual, debe haber 5-6 compuestos por cada descriptor o variable seleccionada.
- 🔢 <u>Justificación de las variables independientes</u>: la mejor ecuación será aquella que presente mejores estadísticos, siendo todos los coeficientes significativamente diferentes de cero.
- 🧘 <u>Principio de Parsimonia</u>: si las reglas anteriores se cumplen para más de un modelo, elegir el más simple.
- 🧠 <u>Modelo final con racionalidad cualitativa</u>: el modelo seleccionado, ¿puede ser explicado o relacionado con el caso en estudio?

## 🧪 **Actividades**
Los estudios de construcción y análisis de los modelos se encuentran previstos en 3 etapas secuenciales:

**1.** Construcción de un modelo de QSAR inicial con un n limitado.\
**2.** Construcción de un modelo de QSAR expandido.\
**3.** Construcción de un modelo de QSAR con más descriptores.

#### **1.** Construcción de un modelo de QSAR inicial con un *n* limitado. 

En el Seminario N°4 seleccionamos un set conformado por 70 agonistas del receptor adrenérgico β2 reportados en la base de datos ChEMBL. Les calculamos 5 descriptores (peso molecular, logP, cantidad de enlaces rotables y número de aceptores y donores de puente hidrógeno), y normalizamos los valores.\
Tomaremos inicialmente una muestra de esas moléculas para introducirnos en QSAR, y familiarizarnos con las reglas de Unger y Hansch.

In [ ]:
from FuncionesTP4 import *

In [ ]:
df_agonistas_B2 = pd.read_csv('../Seminario4/df_beta2_agonists_lipinski_norm.csv') 
itables.show(df_agonistas_B2)

In [ ]:
ids_primer_set = ['CHEMBL679',  'CHEMBL434','CHEMBL4279962', 'CHEMBL462313', 'CHEMBL471','CHEMBL429','CHEMBL49080', 'CHEMBL714', 'CHEMBL1902627','CHEMBL1160723',]#'CHEMBL3747244', 'CHEMBL3746068',
Primer_set = df_agonistas_B2[df_agonistas_B2['molecule_chembl_id'].isin(ids_primer_set)]
itables.show(Primer_set)

**Análisis de correlación intervariables**
\
El primer paso en el estudio estadístico es la realización del análisis de correlación intervariables, de manera de descartar los pares de descriptores que tengan una alta correlación entre sí (*regla 1: independencia de variables*).
\
Para ello, emplearemos una función que lo determinará automáticamente, y generará un mapa de calor (*heatmap*) con los valores de correlación indicados.

📌 Analice el resultado y seleccione los pares de variables que presenten una baja correlación entre sí para que puedan conformar una relación lineal múltiple de acuerdo a las reglas de Hunger-Hansh.

📌 ¿Son lógicos los resultados para aquellos pares de variables que presentan alta correlación? Justifique.

📌 ¿Es aconsejable obtener ecuaciones lineales multiparamétricas? Justifique.

In [ ]:
graficar_heatmap_pearson(Primer_set, columna_inicio='MolWt_norm')

**Confección de las ecuaciones lineales simples**

Analizaremos una por una todas las posibles ecuaciones lineales simples para extraer la información estadística necesaria que nos permita seleccionar el mejor modelo de QSAR del caso en estudio.

Ejecutando la siguiente función, podrán ver los gráficos de regresión, la tabla de resultados con todos los parámetros calculados, la ecuación correspondiente y el valor de R$^{2}$.

In [ ]:
descriptores = ['MolWt_norm', 'MolLogP_norm', 'NumHAcceptors_norm', 'NumHDonors_norm', 'NumRotatableBonds_norm']
resultados = analizar_regresiones_lineales_simples(Primer_set, 'IC50_value_nM_norm', descriptores)
resultados

En base a los resultados:

📌 Identifique y plantee la equación con mayor potencial predictivo, y que cumpla con las cuatro primeras reglas de Unger y Hansch. 

📌 Teniendo en cuenta la información provista respecto del modo de acción los agonistas del receptor adrenérgico β2, analice si los resultados del modelo de QSAR obtenidos son cualitativamente razonables.

#### **2.** Construcción de un modelo de QSAR expandido.

Hemos identificado que usar un número reducido de moléculas, y consecuentemente solo plantear modelos simples o de descriptores únicos, no es una estrategia racional.
Por lo tanto, ahora sí emplearemos el set completo de 70 agonistas. De la misma manera que se ha realizado en la sección anterior y siguiendo las reglas de Unger y Hansch, utilizaremos el conjunto de datos extendidos para construir un nuevo modelo de QSAR, identificando la ecuación con mayor potencial predictivo.

In [ ]:
itables.show(df_agonistas_B2)

📌 Analice el mapa de correlaciones para el nuevo set de compuestos. ¿Hubo algún cambio respecto al set anterior? 

📌 ¿Es aconsejable obtener ecuaciones lineales multiparamétricas? Justifique.

📌 Seleccione los pares de variables que presenten una baja correlación entre sí para que puedan conformar una relación lineal múltiple de acuerdo a las reglas de Hunger-Hansh.

In [ ]:
graficar_heatmap_pearson(df_agonistas_B2, columna_inicio='MolWt_norm')

Cuando aumenta el número de moléculas y, consecuentemente, las combinaciones de descriptores que pueden combinarse en modelos multiparamétricos, el análisis de correlación puede complicarse.
Por lo tanto, la siguiente función permite identificar y generar automáticamente todas las combinaciones posibles de ecuaciones lineales simples y multiparamétricas. 

Ejecútenla y verifiquen el resultado que analizaron anteriormente.

In [ ]:
descriptores = ['MolWt_norm', 'MolLogP_norm', 'NumHAcceptors_norm', 'NumHDonors_norm', 'NumRotatableBonds_norm']
combis_validas = obtener_combinaciones_validas(df_agonistas_B2, descriptores)

print(f"Se encontraron {len(combis_validas)} combinaciones válidas:")
for c in combis_validas:
    print(c)

In [ ]:
df_qsar = evaluar_modelos_qsar(df_agonistas_B2, descriptores, y_col='IC50_value_nM_norm')
itables.show(df_qsar)

En base a los resultados:

📌 Identifique y plantee la equación con mayor potencial predictivo, y que cumpla con las cuatro primeras reglas de Unger y Hansch. 

📌 Teniendo en cuenta la información provista respecto del modo de acción los agonistas del receptor adrenérgico β2, analice si los resultados del modelo de QSAR obtenidos son cualitativamente razonables.

#### **3.** Construcción de un modelo de QSAR con más descriptores.

Tal como aprendieron en los teóricos de QSAR, existe una gran diversidad de descriptores moleculares.

En este sentido, calcularemos ahora nuevos descriptores para el set de agonistas, y repetiremos todo el análisis para intentar identificar un mejor modelo de QSAR.

In [ ]:
nuevos_descriptores = ['TPSA','PEOE_VSA14','SMR_VSA3','RingCount','EState_VSA3','VSA_EState10',
                       'LabuteASA', 'MaxPartialCharge','MolMR','MinPartialCharge','NumAromaticRings']

df_con_descriptores = agregar_descriptores(df_agonistas_B2, seleccion=nuevos_descriptores,antes_de_columna='IC50_value_nM_norm')
itables.show(df_con_descriptores)

Recuerden que es importante normalizar los datos, tal como hicimos con los descriptores de Lipinski.

In [ ]:
df_agonistas_B2_mas_desc = normalizar_columnas_seleccionadas(df_con_descriptores, nuevos_descriptores)
itables.show(df_agonistas_B2_mas_desc)

In [ ]:
def graficar_distribucion_descriptores_all(df, columna_inicio, titulo="Distribución de descriptores moleculares"):
    """
    Genera un gráfico de violín para descriptores moleculares a partir de una columna dada hasta el final del DataFrame.

    Parámetros:
        df (pd.DataFrame): DataFrame con los datos.
        columna_inicio (str): Nombre de la columna desde donde empezar a incluir variables.
        titulo (str): Título del gráfico.
    """
    # Obtener columnas desde 'columna_inicio' hasta el final
    idx_inicio = df.columns.get_loc(columna_inicio)
    variables = df.columns[idx_inicio:]
    
    df_long = df[variables].melt(var_name="Descriptor", value_name="Valor")
    
    plt.figure(figsize=(20, 8))
    sns.violinplot(x="Descriptor", y="Valor", data=df_long)
    plt.xticks(rotation=45)
    plt.title(titulo)
    plt.tight_layout()
    plt.show()

graficar_distribucion_descriptores_all(df_agonistas_B2_mas_desc, 'MolWt_norm')
    

📌 Realice el análisis de correlación y determine todas las ecuaciones posibles.

Luego, en base a los resultados:

📌 Identifique y plantee la equación con mayor potencial predictivo, y que cumpla con las cuatro primeras reglas de Unger y Hansch. 

📌 Analice si los resultados del modelo de QSAR obtenidos son cualitativamente razonables, teniendo en cuenta la información provista respecto del modo de acción los agonistas del receptor adrenérgico β2,

📌 Explique los detalles que diferencian a los modelos construidos en las tres actividades, indicando cuál de ellos considera que presenta el mayor potencial predictivo.

In [ ]:
graficar_heatmap_pearson(df_agonistas_B2_mas_desc, columna_inicio='MolWt_norm')

In [ ]:
descriptores_nuevos = ['MolWt_norm', 'MolLogP_norm', 'NumHAcceptors_norm', 'NumHDonors_norm', 'NumRotatableBonds_norm',
                    'TPSA_norm', 'SMR_VSA3_norm', 'RingCount_norm', 'EState_VSA3_norm', 'LabuteASA_norm',
                    'VSA_EState10_norm', 'MaxPartialCharge_norm','MolMR_norm', 'MinPartialCharge_norm', 'NumAromaticRings_norm']#'PEOE_VSA14_norm',

combis_validas = obtener_combinaciones_validas(df_agonistas_B2_mas_desc, descriptores_nuevos)

print(f"Se encontraron {len(combis_validas)} combinaciones válidas!")
#for c in combis_validas:
#    print(c)

In [ ]:
df_qsar_new = evaluar_modelos_qsar(df_agonistas_B2_mas_desc, descriptores_nuevos, y_col='IC50_value_nM_norm')
itables.show(df_qsar_new)

IC50 = -0.14775(± 0.1184)* *NumHDonors* + 0.2444(±0.1863)* *TPSA* + 0.8861(±0.1478)* *LabuteASA* + 0.4287(±0.0606)* *MinPartialCharge* - 0.408 

**<u>NumHDonors</u>**: Coeficiente negativo => Sugiere que los puentes hidrógeno son necesarios para la unión al receptor β2.

**<u>TPSA</u>** (Topological Polar Surface Area): El área superficial ocupada por átomos polares, relacionada con la capacidad de formar enlaces de hidrógeno.\
Coeficiente positivo => Un exceso de polaridad reduce la permeabilidad o interfiere con los contactos en el sitio activo. Aunque cierta polaridad es necesaria (ej. puentes H, basicidad), demasiada podría dificultar el paso por membranas o afectar la orientación en el sitio de unión.

**<u>LabuteASA</u>** (Approximate Surface Area, calculada por el método de Labute): Área superficial accesible al solvente, ajustada por tipo atómico. ¿Qué tan expuesta está la molécula al entorno acuoso?\
Coeficiente positivo => Refleja que moléculas demasiado hidrofílicas reducen la bioactividad.

**<u>MinPartialCharge</u>**: Mínima carga parcial. Coeficiente positivo => Sugiere que no conviene tener zonas de carga excesivamente negativa, acorde con el hecho de que el receptor β2 tiene zonas negativas en su pocket y los ligandos son bases.


Si quisieran usar las primeras...

**<u>EState_VSA</u>**: Área superficial asociada a átomos con cierto valor electrónico (índices topológicos y electrónicos combinados). ¿Qué tan electrónicamente activos son ciertos átomos en una región? ¿Cuán electrónicos son los átomos con cierta superficie?

**<u>VSA\_EState</u>**: Similar a EStateVSA, pero agrupado según valores de EState en otro rango específico. ¿Cuál es la distribución electrónica superficial?  ¿Cuánta superficie total tienen los átomos con cierto valor electrónico?

Ambas con coeficientes negativos => Regiones con mayor densidad electrónica  favorecen interacciones electrostáticas con residuos del receptor. O superficies activas electrónicamente, mejoran actividad. Sugiere que regiones electrónicamente ricas están involucradas en interacciones clave, como donación de electrones (bases) o puentes H.


